In [ ]:
!pip install ntscraper
import tweepy
import pandas as pd
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from ntscraper import Nitter
scraper = Nitter(log_level = 1,skip_instance_check = False)
tweets = scraper.get_tweets("BJP4India",mode = "user",number = 100)
from pprint import pprint
pprint(tweets)
tweets.keys()
tweets['tweets']


data ={
    'link':[],
    'text':[],
    'user':[],
    'likes':[],
    'quotes':[],
    'retweets':[],
    'comments':[]
}

for tweet in tweets['tweets']:
  data['link'].append(tweet['link'])
  data['text'].append(tweet['text'])
  data['user'].append(tweet['user'])
  data['likes'].append(tweet['stats']['likes'])
  data['quotes'].append(tweet['stats']['quotes'])
  data['retweets'].append(tweet['stats']['retweets'])
  data['comments'].append(tweet['stats']['comments'])

import pandas as pd
df =pd.DataFrame(data)
df.head()


#Data Cleaning is required
# Create a function
def cleanTxt(text):
  text=re.sub(r'@[A-Za-z0-9]+','',text) # removes @mentions text=re.sub(r'#', '', text)
  text=re.sub(r'RT[\s]+','',text)
  text=re.sub(r'https?:\/\/\S+', '',text)
  return text
df['text']= df['text'].apply(cleanTxt)

#show cleaned data
df




def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#Create 2 more columns to store subjectivity and polarity
df['Subjectivity'] = df['text'].apply(getSubjectivity)
df['Polarity']= df['text'].apply(getPolarity)

#Show the new data frames with new columns
df





# Word Cloud Plotting ,
allWords =' '.join([twts for twts in df['text']])
wordCloud = WordCloud(width =500, height=300, random_state=21, max_font_size=119).generate(allWords)
plt.imshow(wordCloud, interpolation ="bilinear")
plt.axis('off')
plt.show



#Create function to compute negate , neutral and positive analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'
df['Analysis'] =df['Polarity'].apply(getAnalysis)

#Show dataframe
df





#Print all positive tweets
j=1
sortedDF= df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i]=='Positive'):
    print(str(j)+')' +sortedDF['text'][i])
    print()
    j=j+1



#Print all Negative tweets
j=1
sortedDF= df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i]=='Negative'):
    print(str(j)+')' +sortedDF['text'][i])
    print()
    j=j+1




#Print all neutral tweets
j=1
sortedDF= df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i]=='Neutral'):
    print(str(j)+')' +sortedDF['text'][i])
    print()
    j=j+1




# Plot Polarity and Subjectivity
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color='Blue')
plt.title("Sentiment Analysis")
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()


Testing instances:  36%|███▋      | 28/77 [00:29<00:17,  2.76it/s]